In [142]:
import altair as alt
from cabi_functions import load_stations
import geopandas as gpd
import pandas as pd
import requests
from shapely.ops import orient

In [143]:
dc_wards = gpd.read_file('https://opendata.arcgis.com/datasets/25b86859702141cb840eb50136ebc396_50.geojson')
dc_wards.geometry = dc_wards.geometry.apply(orient, args=(-1,))

In [144]:
basechart = alt.Chart(dc_wards).mark_geoshape(fill='lightgray',stroke='gray').properties(
    width=750*2,
    height=500*2
    )

In [145]:
# Define which attributes to lookup from airports.csv
cabi_stations = 'https://raw.githubusercontent.com/mlinds/cabi-data/main/data/stationLookup.csv'
connections_path = 'https://raw.githubusercontent.com/mlinds/cabi-data/main/data/connections_csv.csv'


In [146]:
lookup_data = alt.LookupData(
    cabi_stations, key="short_name", fields=["name", "lat", "lon"]
)

lookup_pop = alt.LookupData(
    connections_path, key="startid", fields=["popularity"]
)

In [147]:
# Create mouseover selection
select_station = alt.selection_single(
    on="mouseover", nearest=True, fields=["short_name","startid"], empty="none"
)

In [150]:
connections = alt.Chart(connections_path).mark_rule().encode(
    latitude="lat:Q",
    longitude="lon:Q",
    latitude2="lat2:Q",
    longitude2="lon2:Q"
).transform_lookup(
    lookup="startid",
    from_=lookup_data
).transform_lookup(
    lookup="endid",
    from_=lookup_data,
    as_=["name2", "lat2", "lon2"]
)
# .transform_filter(
#     select_station
# )


points = alt.Chart(cabi_stations).mark_circle().encode(
    latitude = 'lat:Q',
    longitude = 'lon:Q',
    tooltip=["name:N","short_name:Q","lat:Q","lon:Q"]
).transform_lookup(
    lookup = 'short_name',
    from_ = lookup_pop
).add_selection(
    select_station
)


In [151]:
basechart + points + connections

alt.LayerChart(...)